In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

from matplotlib import rcParams
rcParams["savefig.dpi"] = 200
rcParams["font.size"] = 8

import warnings
warnings.filterwarnings('ignore')

# Create xarray region mask

In this tutorial we will show how to create a mask for arbitrary latitude and longitude grids using xarray. It is very similar to the tutorial Create Mask (numpy).

Import regionmask and check the version:

In [ ]:
import regionmask
regionmask.__version__

Load xarray and the tutorial data:

In [ ]:
import xarray as xr
import numpy as np

In [ ]:
airtemps = xr.tutorial.load_dataset('air_temperature')

The example data is a temperature field over North America. Let's plot the first time step:

In [ ]:
# load plotting libraries
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# choose a good projection for regional maps
proj=ccrs.LambertConformal(central_longitude=-100)

ax = plt.subplot(111, projection=proj)

airtemps.isel(time=1).air.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree())

ax.coastlines();

Conviniently we can directly pass an xarray object to the `mask` function. It gets the longitude and latitude from the DataArray/ Dataset and creates the `mask`. If the longituda and latitude in the xarray object are not called `lon` and `lat`, respectively; their name can be given via the `lon_name` and `lat_name` keyword. Here we use the Giorgi regions.

The extent of the grid and the regions is not the same:

In [ ]:
lon = airtemps.lon
print("Grid extent:    ", lon.values.min(), lon.values.max())

bounds = regionmask.defined_regions.giorgi.bounds_global
print("Region extent: ", bounds[0], bounds[2])

In [ ]:
mask = regionmask.defined_regions.giorgi.mask(airtemps)

This did not work in earlier versions. We had to set `wrap_lon=True`.

Let's plot the regions:

In [ ]:
proj=ccrs.LambertConformal(central_longitude=-100)
ax = plt.subplot(111, projection=proj)

low = mask.min()
high = mask.max()

levels = np.arange(low - 0.5, high + 1)

mask.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree(), levels=levels, add_colorbar=False)

ax.coastlines()

# fine tune the extent
ax.set_extent([200, 330, 10, 75], crs=ccrs.PlateCarree());

We want to select the region 'Central North America'. Thus we first need to find out which number this is:

In [ ]:
regionmask.defined_regions.giorgi.map_keys('Central North America')

## Select using `where`

`xarray` provides the handy `where` function:

In [ ]:
airtemps_CNA = airtemps.where(mask == 6)

Check everything went well by repeating the first plot with the selected region:

In [ ]:
# choose a good projection for regional maps
proj=ccrs.LambertConformal(central_longitude=-100)

ax = plt.subplot(111, projection=proj)

airtemps_CNA.isel(time=1).air.plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree())

ax.coastlines();

Looks good - let's take the area average and plot the time series. (Note: you should use `cos(lat)` weights to correctly calculate an area average. Unfortunately this is not yet (as of version 0.13) implemented in xarray.) 

In [ ]:
ts_airtemps_CNA = airtemps_CNA.mean(dim=('lat', 'lon')) - 273.15
ts_airtemps = airtemps.mean(dim=('lat', 'lon')) - 273.15

# and the line plot
f, ax = plt.subplots()
ts_airtemps_CNA.air.plot.line(ax=ax, label='Central North America')
ts_airtemps.air.plot(ax=ax, label='Entire Domain')

plt.legend(ncol=2);

## Select using `groupby`

In [ ]:
# xarray version > 0.8 is required
xr.__version__

In [ ]:
# you can group over all integer values of the mask
# you have to take the mean over `stacked_lat_lon`
airtemps_all = airtemps.groupby(mask).mean('stacked_lat_lon')

we can add the abbreviations and names of the regions to the DataArray

In [ ]:
# extract the abbreviations and the names of the regions from regionmask
abbrevs = regionmask.defined_regions.giorgi[airtemps_all.region.values].abbrevs
names = regionmask.defined_regions.giorgi[airtemps_all.region.values].names

airtemps_all.coords['abbrevs'] = ('region', abbrevs)
airtemps_all.coords['names'] = ('region', names)

In [ ]:
airtemps_all

now we can select the regions in many ways

In [ ]:
# as before, by the index of the region
r1 = airtemps_all.sel(region=6).air

# with the abbreviation
r2 = airtemps_all.isel(region=(airtemps_all.abbrevs == 'WNA')).air.squeeze()

# with the long name
r3 = airtemps_all.isel(region=(airtemps_all.names == 'Eastern North America')).air.squeeze()

regs = [r1, r2, r3]

Now, let's plot the three selected regions:

In [ ]:
f, axes = plt.subplots(3, 1, sharex=True)

for i, reg in enumerate(regs):
    ax = axes[i]
    reg.plot(ax=ax)
    ax.set_title(reg.names.values)

plt.setp(axes, xlabel="")

f.tight_layout()